<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [18]:
%%bash

# rm -r sample_data 
# git clone https://github.com/wyattowalsh/sports-analytics.git

# # # install conda
# # bash sports-analytics/project_resources/bash_scripts/install_conda_in_colab.sh 

# # install dependencies
pip install -r ../../dependencies/basketball/data_collection.txt 

  Using cached nba_api-1.1.9-py3-none-any.whl (242 kB)
  Using cached python_slugify-4.0.1-py2.py3-none-any.whl
  Using cached text_unidecode-1.3-py2.py3-none-any.whl (78 kB)
  Created wheel for kaggle: filename=kaggle-1.5.12-py3-none-any.whl size=73053 sha256=d16e095bb760bafab595115f2389c43aa5ecbf0b5258234f52d746705733e6e6
  Stored in directory: /Users/wyattowalsh/Library/Caches/pip/wheels/ac/b2/c3/fa4706d469b5879105991d1c8be9a3c2ef329ba9fe2ce5085e
Successfully built kaggle


In [77]:
from nba_api.stats.static import players, teams
from nba_api.stats.endpoints import commonplayerinfo, playercareerstats
import pandas as pd 
import numpy as np
import os
import sqlite3 as sql
import matplotlib.pyplot as plt
import seaborn
import multiprocessing as mp
import time

# !mkdir -p ~/.kaggle/ && mv < your kaggle json path > ~/.kaggle/ && chmod 600 ~/.kaggle/kaggle.json
import kaggle

In [31]:
conn = sql.connect('../data/basketball.sqlite')

In [24]:
df_players = pd.DataFrame(players.get_players()).astype({'id': 'str'})
df_players

,id,full_name,first_name,last_name,is_active
0,76001,Alaa Abdelnaby,Alaa,Abdelnaby,False
1,76002,Zaid Abdul-Aziz,Zaid,Abdul-Aziz,False
2,76003,Kareem Abdul-Jabbar,Kareem,Abdul-Jabbar,False
3,51,Mahmoud Abdul-Rauf,Mahmoud,Abdul-Rauf,False
4,1505,Tariq Abdul-Wahad,Tariq,Abdul-Wahad,False
...,...,...,...,...,...
4496,1627790,Ante Zizic,Ante,Zizic,True
4497,78647,Jim Zoet,Jim,Zoet,False
4498,78648,Bill Zopf,Bill,Zopf,False
4499,1627826,Ivica Zubac,Ivica,Zubac,True


In [25]:
df_players.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4501 entries, 0 to 4500
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   id          4501 non-null   object
 1   full_name   4501 non-null   object
 2   first_name  4501 non-null   object
 3   last_name   4501 non-null   object
 4   is_active   4501 non-null   bool  
dtypes: bool(1), object(4)
memory usage: 145.2+ KB


In [26]:
df_players.to_sql('Player', conn)

In [58]:
df_teams = pd.DataFrame(teams.get_teams()).astype({'id': 'str'})
df_teams['year_founded'] =  pd.to_datetime(df_teams['year_founded'], format='%Y').dt.year
df_teams

,id,full_name,abbreviation,nickname,city,state,year_founded
0,1610612737,Atlanta Hawks,ATL,Hawks,Atlanta,Atlanta,1949
1,1610612738,Boston Celtics,BOS,Celtics,Boston,Massachusetts,1946
2,1610612739,Cleveland Cavaliers,CLE,Cavaliers,Cleveland,Ohio,1970
3,1610612740,New Orleans Pelicans,NOP,Pelicans,New Orleans,Louisiana,2002
4,1610612741,Chicago Bulls,CHI,Bulls,Chicago,Illinois,1966
5,1610612742,Dallas Mavericks,DAL,Mavericks,Dallas,Texas,1980
6,1610612743,Denver Nuggets,DEN,Nuggets,Denver,Colorado,1976
7,1610612744,Golden State Warriors,GSW,Warriors,Golden State,California,1946
8,1610612745,Houston Rockets,HOU,Rockets,Houston,Texas,1967
9,1610612746,Los Angeles Clippers,LAC,Clippers,Los Angeles,California,1970


In [28]:
df_teams.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30 entries, 0 to 29
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   id            30 non-null     object
 1   full_name     30 non-null     object
 2   abbreviation  30 non-null     object
 3   nickname      30 non-null     object
 4   city          30 non-null     object
 5   state         30 non-null     object
 6   year_founded  30 non-null     int64 
dtypes: int64(1), object(6)
memory usage: 1.8+ KB


In [29]:
df_teams.to_sql('Team', conn)

In [ ]:
player_ids = df_players['id'].values
res = commonplayerinfo.CommonPlayerInfo(player_id=player_ids[0])
dfs = res.get_data_frames()
Player_Attributes = pd.merge(dfs[0], dfs[1], how='left', left_on=['PERSON_ID', 'DISPLAY_FIRST_LAST'], right_on=['PLAYER_ID', 'PLAYER_NAME'])
Player_Attributes = Player_Attributes.drop(['TimeFrame'], axis=1)

def f(player_id):
    res = commonplayerinfo.CommonPlayerInfo(player_id=player_id)
    dfs = res.get_data_frames()
    df = pd.merge(dfs[0], dfs[1], how='left', left_on=['PERSON_ID', 'DISPLAY_FIRST_LAST'], right_on=['PLAYER_ID', 'PLAYER_NAME'])
    df = df.drop(['TimeFrame'], axis=1)
    time.sleep(3)
    return(df)

if __name__ == '__main__':
    N = mp.cpu_count()

    with mp.Pool(processes = N) as p:
        common_player_info = p.map(f, player_ids)

        
        
# # for player_id in player_ids[1:]: 
# #     res = commonplayerinfo.CommonPlayerInfo(player_id=player_id)
# #     dfs = res.get_data_frames()
# #     df = pd.merge(dfs[0], dfs[1], how='left', left_on=['PERSON_ID', 'DISPLAY_FIRST_LAST'], right_on=['PLAYER_ID', 'PLAYER_NAME'])
# #     df = df.drop(['TimeFrame'], axis=1)
# #     Player_Attributes = pd.concat([Player_Attributes, df])
# # print(len(Player_Attributes))
# # Player_Attributes

In [76]:
Player_Attributes

,PERSON_ID,FIRST_NAME,LAST_NAME,DISPLAY_FIRST_LAST,DISPLAY_LAST_COMMA_FIRST,DISPLAY_FI_LAST,PLAYER_SLUG,BIRTHDATE,SCHOOL,COUNTRY,...,GAMES_PLAYED_FLAG,DRAFT_YEAR,DRAFT_ROUND,DRAFT_NUMBER,PLAYER_ID,PLAYER_NAME,PTS,AST,REB,ALL_STAR_APPEARANCES
0,76001,Alaa,Abdelnaby,Alaa Abdelnaby,"Abdelnaby, Alaa",A. Abdelnaby,alaa-abdelnaby,1968-06-24T00:00:00,Duke,USA,...,Y,1990,1,25,76001,Alaa Abdelnaby,5.7,0.3,3.3,0


In [67]:
pd.concat([Player_Attributes, Player_Attributes])

,PERSON_ID,FIRST_NAME,LAST_NAME,DISPLAY_FIRST_LAST,DISPLAY_LAST_COMMA_FIRST,DISPLAY_FI_LAST,PLAYER_SLUG,BIRTHDATE,SCHOOL,COUNTRY,...,GAMES_PLAYED_FLAG,DRAFT_YEAR,DRAFT_ROUND,DRAFT_NUMBER,PLAYER_ID,PLAYER_NAME,PTS,AST,REB,ALL_STAR_APPEARANCES
0,76001,Alaa,Abdelnaby,Alaa Abdelnaby,"Abdelnaby, Alaa",A. Abdelnaby,alaa-abdelnaby,1968-06-24T00:00:00,Duke,USA,...,Y,1990,1,25,76001,Alaa Abdelnaby,5.7,0.3,3.3,0
0,76001,Alaa,Abdelnaby,Alaa Abdelnaby,"Abdelnaby, Alaa",A. Abdelnaby,alaa-abdelnaby,1968-06-24T00:00:00,Duke,USA,...,Y,1990,1,25,76001,Alaa Abdelnaby,5.7,0.3,3.3,0


In [70]:
Player_Attributes.info

<bound method DataFrame.info of    PERSON_ID FIRST_NAME  LAST_NAME DISPLAY_FIRST_LAST  \
0      76001       Alaa  Abdelnaby     Alaa Abdelnaby   

  DISPLAY_LAST_COMMA_FIRST DISPLAY_FI_LAST     PLAYER_SLUG  \
0          Abdelnaby, Alaa    A. Abdelnaby  alaa-abdelnaby   

             BIRTHDATE SCHOOL COUNTRY  ... GAMES_PLAYED_FLAG DRAFT_YEAR  \
0  1968-06-24T00:00:00   Duke     USA  ...                 Y       1990   

  DRAFT_ROUND  DRAFT_NUMBER PLAYER_ID     PLAYER_NAME  PTS  AST  REB  \
0           1            25     76001  Alaa Abdelnaby  5.7  0.3  3.3   

  ALL_STAR_APPEARANCES  
0                    0  

[1 rows x 38 columns]>

In [ ]:
Player_Attributes